# 特別サイズトライアルと店舗決済売上upliftの検証

## Import libraries

In [1]:
!pip install pycausalimpact

In [0]:
from matplotlib import colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from causalimpact import CausalImpact

## set seed

In [0]:
import random
random.seed(1)

## Mout Google Drive

In [4]:
from google.colab import drive
drive.mount('/Drive')

Drive already mounted at /Drive; to attempt to forcibly remount, call drive.mount("/Drive", force_remount=True).


## Local packages

In [5]:
cd /Drive/My\ Drive/working_space

/Drive/My Drive/working_space


In [0]:
import robust_synth
import synth_functions 

## 関数定義

In [0]:
def synth_plots_bayes(obs, bayes, sigma_hat, title, xlabel, ylabel, region, year, year_shift, loc, up_lim):
    fig, ax = plt.subplots()
    ax.plot(obs, label=region, linewidth=1.75, color='k')
    ax.plot(bayes, '--', label='Synthetic ' + region + " (Bayes)", linewidth=1.75, color='b')
    x_ = np.linspace(0, len(bayes) - 1, len(bayes))
    clr1 = 'lightcyan'
    clr2 = 'paleturquoise'
    upper = bayes + sigma_hat
    lower = bayes - sigma_hat
    ax.fill_between(x_, bayes, upper, facecolor=clr1, edgecolor=clr2, interpolate=True)
    ax.fill_between(x_, bayes, lower, facecolor=clr1, edgecolor=clr2, interpolate=True)
    legend = ax.legend(loc=loc, shadow=True, prop={'size': 9.5})
    frame = legend.get_frame()
    frame.set_facecolor('0.925')
    ax.plot([year, year], [0, up_lim], '--', linewidth=1.5, color='r')
    years = int(np.floor(obs.shape[0] / 5))
    x = np.array([5 * i for i in range(years + 1)])
    ax.set_ylim([0, up_lim])
    plt.xticks(x, x + year_shift)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()


def synth_plots(obs, linear, ridge, lasso, title, xlabel, ylabel, region, year, year_shift, loc, upper):
    fig, ax = plt.subplots()
    ax.plot(obs, label=region, linewidth=1.75, color='k')
    ax.plot(linear, '--', label='Synthetic ' + region + " (linear)", linewidth=1.75, color='b')
    ax.plot(ridge, '--', label='Synthetic ' + region + " (ridge)", linewidth=1.75, color='g')
    ax.plot(lasso, '--', label='Synthetic ' + region +
            " (lasso)", linewidth=1.75, color='darkorange')
    legend = ax.legend(loc=loc, shadow=True, prop={'size': 9.5})
    frame = legend.get_frame()
    frame.set_facecolor('0.925')
    ax.plot([year, year], [0, upper], '--', linewidth=1.5, color='r')
    years = int(np.floor(obs.shape[0] / 5))
    x = np.array([5 * i for i in range(years + 1)])
    ax.set_ylim([0, upper])
    plt.xticks(x, x + year_shift)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()


def plot_loop():
    plt.rcParams["figure.figsize"] = [12,10]
    for target in treatment_store:

        synth_df_target = synth_df_treated[synth_df_treated['store_name'] == target]
        test_df = pd.concat([synth_df_target, synth_df_control], axis = 0).reset_index(drop = True)
        del test_df['treatment']
        synth_test = test_df.pivot_table(values = 'store_register_order_value_log', index = 'store_name', columns = 'time_index_int')

        ## おまじない
        synth_test.dropna(inplace=True)

        ## skip
        if target not in list(synth_test.index):
            continue

        ##-------------------------------------------------
        ## Given variables
        ##-------------------------------------------------
        print(target)
        time_shift = 1
        xlabel = "time_index"
        ylabel = "store_register_order_value_log"
        title = 'test synthetic control'

        linear = robust_synth.Synth(target, year = intervention , num_sv = 2, method = "Linear")
        ridge = robust_synth.Synth(target, year = intervention, num_sv = 2, method = "Ridge")
        lasso = robust_synth.Synth(target, year = intervention, num_sv = 2, method = "Lasso")

        linear.fit(synth_test)
        ridge.fit(synth_test)
        lasso.fit(synth_test)
        obs = linear.orig  # the actual observed trajectory

        # predicted means (pre + post intervention) using linear/ridge/lasso regression
        linear_mean = linear.mean
        ridge_mean = ridge.mean
        lasso_mean = lasso.mean

        synth_plots(obs=obs, linear=linear_mean, ridge=ridge_mean, lasso=lasso_mean, title=title, xlabel=xlabel,
                    ylabel=ylabel, region= target, year=intervention, year_shift=time_shift, loc="lower right", upper = max(obs)*1.2)
        
        ##-------------------------------------------------
        ## Bayes
        ##-------------------------------------------------
        for n in range(4, 7):
            bayes = robust_synth.Synth(target, year=intervention, method="bayes", num_sv=n, prior_param=10)
            bayes.fit(synth_test)
            obs = bayes.orig
            bayes_mean = bayes.mean
            title = "synthetic test: # singular values = " + str(n)
        
        synth_plots_bayes(obs=obs, bayes=bayes_mean, sigma_hat=bayes.sigma_hat, title=title, xlabel=xlabel,
                             ylabel=ylabel, region=target, year=intervention, year_shift=time_shift, loc="lower right", up_lim=15)
        
        data = pd.DataFrame({'y': obs, 'X': bayes_mean}, columns=['y', 'X'])
        ci = CausalImpact(data, [0,intervention - 1], [intervention, len(obs)-1])
        print(ci.summary())
        ci.plot()

### dataの取り込み

In [8]:
## change working directory
cd /Drive/My\ Drive/redshift_data

/Drive/My Drive/redshift_data


In [9]:
## fileの存在確認
%%bash
FILE=weekly_dataframe_synthetic_control_20200203.csv
if test -f "$FILE"; then
    echo "$FILE exist"
fi

weekly_dataframe_synthetic_control_20200203.csv exist


### カラム名の抽出
正規表現を用いてSQL FILEから抽出
```
検索：(.*)\sAS\s(\w*)
置換：$2
```
その後
```
検索：^
置換:'
```
Then,
```
検索：,\n
置換:', 
```




In [10]:
FILE_PATH = 'weekly_dataframe_synthetic_control_20200203.csv'
df = pd.read_csv(FILE_PATH, header = None, names = ('loc_idnt', 'store_name', 'treatment', 'time_index','store_order_value', 'store_order_cnt', 'kantan_order_value', 'kantan_order_cnt', 'kantan_ec_special_order_value', 'kantan_ec_special_order_cnt', 'store_register_order_value',
                                                      'store_register_order_cnt', 'store_register_ec_special_order_value',
                                                      'store_register_ec_special_order_cnt'))
df.head()

,loc_idnt,store_name,treatment,time_index,store_order_value,store_order_cnt,kantan_order_value,kantan_order_cnt,kantan_ec_special_order_value,kantan_ec_special_order_cnt,store_register_order_value,store_register_order_cnt,store_register_ec_special_order_value,store_register_ec_special_order_cnt
0,100119,姪浜店,0,2019-10-12 00:00:00.000000,22271772,5405,21530,6,10970,3,96880,23,70220,12
1,100570,日立成沢店,0,2019-11-30 00:00:00.000000,17948039,4144,71590,19,14130,5,123870,34,38650,13
2,100617,広島八木店,0,2019-10-05 00:00:00.000000,17477186,4605,46420,11,25200,4,69510,18,29770,6
3,100699,十和田店,0,2019-10-26 00:00:00.000000,11182903,2490,14750,6,11960,4,25020,9,18530,6
4,100863,柏崎店,0,2019-10-12 00:00:00.000000,11149696,2940,27940,13,10550,4,48850,20,25300,9


## 前処理
### 時間の処理
1. time_indexのユニークキーからなるリストを作成
2. 時間順にsortする
3. 順番と対応したdictを作成
4. dictに従ってdataframeのtime_indexを数値に変更したカラム`time_index_int`を作成

### 異常な値をとる店舗の削除
- 1期間以上営業していないと認められる店舗は除外
- 店舗売上が0の期間があるかないか判断
- 店舗決済が0

In [0]:
## dict作成
time_set = list(set(df.time_index))
time_set.sort()
time_order = [i for i in range(1, len(time_set) + 1)]
time_dict = dict(zip(time_set, time_order))
intervention = time_dict['2019-11-30 00:00:00.000000'] - 1
#time_dict

In [0]:
## time_index_int
df['time_index_int'] = df['time_index'].map(time_dict)

In [13]:
df.head()

,loc_idnt,store_name,treatment,time_index,store_order_value,store_order_cnt,kantan_order_value,kantan_order_cnt,kantan_ec_special_order_value,kantan_ec_special_order_cnt,store_register_order_value,store_register_order_cnt,store_register_ec_special_order_value,store_register_ec_special_order_cnt,time_index_int
0,100119,姪浜店,0,2019-10-12 00:00:00.000000,22271772,5405,21530,6,10970,3,96880,23,70220,12,7
1,100570,日立成沢店,0,2019-11-30 00:00:00.000000,17948039,4144,71590,19,14130,5,123870,34,38650,13,14
2,100617,広島八木店,0,2019-10-05 00:00:00.000000,17477186,4605,46420,11,25200,4,69510,18,29770,6,6
3,100699,十和田店,0,2019-10-26 00:00:00.000000,11182903,2490,14750,6,11960,4,25020,9,18530,6,9
4,100863,柏崎店,0,2019-10-12 00:00:00.000000,11149696,2940,27940,13,10550,4,48850,20,25300,9,7


#### 異常な値をとる店舗の削除

In [0]:
drop_candidate = list(set(df[df['store_order_value'] < 1].loc_idnt.values))
drop_index = df[df['loc_idnt'].isin(drop_candidate)].index
df_drop_zeros = df.drop(index = drop_index)

del drop_candidate, drop_index

In [0]:
## 店舗決済件数で異常値をとる店舗の削除
drop_candidate = list(df_drop_zeros[df_drop_zeros['store_register_order_cnt'] < 1].loc_idnt.values)
drop_index = df_drop_zeros[df_drop_zeros['loc_idnt'].isin(drop_candidate)].index
df_drop_zeros = df_drop_zeros.drop(index = drop_index)

del drop_candidate, drop_index

In [16]:
df_drop_zeros.describe()

,loc_idnt,treatment,store_order_value,store_order_cnt,kantan_order_value,kantan_order_cnt,kantan_ec_special_order_value,kantan_ec_special_order_cnt,store_register_order_value,store_register_order_cnt,store_register_ec_special_order_value,store_register_ec_special_order_cnt,time_index_int
count,17344.000000,17344.000000,1.734400e+04,17344.000000,17344.000000,17344.000000,17344.000000,17344.000000,1.734400e+04,17344.000000,17344.000000,17344.000000,17344.000000
mean,103588.202837,0.056792,2.185078e+07,5284.017182,54080.065440,15.406654,24499.313480,5.779809,1.262015e+05,33.962408,59384.375750,13.674239,12.028252
std,4666.835810,0.231451,1.879295e+07,3861.104778,45617.076857,11.495203,24038.876207,4.543308,1.202733e+05,26.042153,61848.835755,10.627549,6.627893
min,100119.000000,0.000000,1.448483e+06,370.000000,0.000000,0.000000,0.000000,0.000000,9.900000e+02,1.000000,0.000000,0.000000,1.000000
25%,100876.000000,0.000000,1.031358e+07,2798.000000,22230.000000,7.000000,7980.000000,3.000000,5.098000e+04,16.000000,21470.000000,6.000000,6.000000
50%,101215.000000,0.000000,1.722801e+07,4377.000000,41945.000000,13.000000,17950.000000,5.000000,9.344500e+04,28.000000,41750.000000,11.000000,12.000000
75%,104090.000000,0.000000,2.750732e+07,6657.000000,73340.000000,20.000000,33660.000000,8.000000,1.615600e+05,44.000000,74650.000000,18.000000,18.000000
max,118508.000000,1.000000,3.027895e+08,55184.000000,524765.000000,116.000000,324325.000000,45.000000,2.800890e+06,445.000000,997260.000000,144.000000,23.000000


## TEST: store_order_value

In [17]:
synth_df = df_drop_zeros.copy()
synth_df = synth_df.reset_index(drop = True)
synth_df.head()

,loc_idnt,store_name,treatment,time_index,store_order_value,store_order_cnt,kantan_order_value,kantan_order_cnt,kantan_ec_special_order_value,kantan_ec_special_order_cnt,store_register_order_value,store_register_order_cnt,store_register_ec_special_order_value,store_register_ec_special_order_cnt,time_index_int
0,100119,姪浜店,0,2019-10-12 00:00:00.000000,22271772,5405,21530,6,10970,3,96880,23,70220,12,7
1,100570,日立成沢店,0,2019-11-30 00:00:00.000000,17948039,4144,71590,19,14130,5,123870,34,38650,13,14
2,100617,広島八木店,0,2019-10-05 00:00:00.000000,17477186,4605,46420,11,25200,4,69510,18,29770,6,6
3,100699,十和田店,0,2019-10-26 00:00:00.000000,11182903,2490,14750,6,11960,4,25020,9,18530,6,9
4,100863,柏崎店,0,2019-10-12 00:00:00.000000,11149696,2940,27940,13,10550,4,48850,20,25300,9,7


不必要なカラムの除去

In [18]:
col_list = ['store_name', 'treatment', 'time_index_int', 'store_register_order_value']
synth_df = synth_df.loc[:, col_list]
synth_df.head()

,store_name,treatment,time_index_int,store_register_order_value
0,姪浜店,0,7,96880
1,日立成沢店,0,14,123870
2,広島八木店,0,6,69510
3,十和田店,0,9,25020
4,柏崎店,0,7,48850


log変換

In [19]:
synth_df['store_register_order_value_log'] = synth_df['store_register_order_value'].apply(lambda x: np.log(x))
del synth_df['store_register_order_value'] ## drop colum

synth_df.head()

,store_name,treatment,time_index_int,store_register_order_value_log
0,姪浜店,0,7,11.481228
1,日立成沢店,0,14,11.726988
2,広島八木店,0,6,11.149226
3,十和田店,0,9,10.127431
4,柏崎店,0,7,10.796510


## Synthetic control実践

## データ処理
1. treatment_store listの作成
2. synth_df_control, synth_df_treatedに分割

In [0]:
### treatment storeの抽出
treatment_store = list(set(synth_df[synth_df['treatment'] == 1].store_name.values))
treatment_store.sort()
# print(len(treatment_store)), 43

In [0]:
##--------------------------------------------
## synth_df_control, synth_df_treatedに分割
##--------------------------------------------
synth_df_control = synth_df[synth_df['treatment'] == 0].copy()
synth_df_treated = synth_df[synth_df['treatment'] == 1].copy()

## 以下loop処理

In [22]:
## warning off
from warnings import filterwarnings
filterwarnings('ignore')

plot_loop()

Output hidden; open in https://colab.research.google.com to view.